In [1]:
import numpy as np
import pandas as pd
from ripser import ripser

# dionysus is used to compute the persistent homology of upper/lower level sets, 
# and I don't think this library works on windows computers. Try macos/linux.
import dionysus as d

# Time series classification via Topological Data Analysis

**To read the paper, please go to** https://doi.org/10.1016/j.eswa.2021.115326

#### Contents

- In Sections 1 and 2, I describe the problem, and explain why subwindowing is important.

- In Section 3, I give the codes the functions.

- In Section 4, I create features from a random time series.

## Section 1: Sliding windows

In a typical time series classification task, we are given a bunch of labeled time series of equal length, and we want to build a model that correctly identifies the class of an unlabeled time series.

On the other hand, sometimes we have only one (or a few) time series from each class, not necessarily of equal duration. If each time series is sufficiently long, we can still generate multiple time series from each of them via the method of **sliding windows**.

Assume that we have a time series `A = [a_0, a_1, a_2, ..., a_n]` where `n` is large enough. The `sliding_windows` function (below) creates equally sized windows from `A`. For instance,

`sliding_windows(A, 4, 2) = [[a_0,a_1,a_2,a_4], [a_2,a_3,a_4,a_5], ..., ]`

![sliding_windows](im1.png)

**Figure 1:** Sliding windows from a signal.

In [2]:
def sliding_windows(y, window_length, window_shift):
    return [y[i:i+window_length] for i in range(0, len(y)-window_length+1, window_shift)]

The choice of `window_size` depends on the problem. For example, if you are doing a stress detection task using ECG (electrocardiogram) signals, a window of $2$ seconds is too small to identify stress. On the other hand, a window of $10$ minutes is way too large, because nobody wants to wait that long. A common choice (for stress detection) is $60$ seconds. (Note that `window_size` will depend on the sampling frequency of the signal.)

The value of `window_shift` is more of a computational issue. A smaller value of `window_shift` creates more windows.

**Caution:** If the `window_shift` is smaller than the `window_size` (which is usually the case), then the windows are overlapping. In order to avoid data leakage between train-test sets, make sure you do train-test split before windowing.

## Section 2: Subwindowing

Let's continue with our example above. Generally, after creating the $60$-second sliding windows, a machine learning scientist tries to create some features on each window. The problem is, creating topological features from a long window is computationally expensive. Furthermore, some topological features are very sensitive to noise (e.g. coughing during ECG measurement). So, we can do the following:
1. create **subwindows** inside each window
2. compute features on each subwindow
3. compute the window features by finding the mean and standard deviation of subwindow features.

![subwindowing](im2.png)

**Figure 2:** The subwindowing method.

### An idea!

In the above figure, subwindow 2, ..., subwindow 5 are shared by Window 1 and Window 2. Why should we compute the features from **the same** subwindows over and over again?

**Answer:** We shouldn't! So, we first need to find subwindow features, then compute the mean and standard deviations to obtain features for the windows.

## Section 3: The functions

### Function 1: `sliding_windows`

The `sliding_windows` function will be used to create the **subwindows**. We have already given it above, but let's write it again.

In [3]:
def sliding_windows(y, subwindow_length, subwindow_shift):
    return [y[i:i+subwindow_length] for i in range(0, len(y)-subwindow_length+1, subwindow_shift)]

### Function 2: `level_set_persistent_homology`

This function is used to compute the persistent homology of upper/lower level sets. For upper level sets, births are greater than deaths.

In [4]:
def level_set_persistent_homology(subwindow, upper = False):
    # from the documentation
    f_lower_star = d.fill_freudenthal(np.array(subwindow).astype('f4'), reverse = upper)
    p = d.homology_persistence(f_lower_star)
    dgms = d.init_diagrams(p, f_lower_star)
    return np.array([[x.birth, x.death] for x in dgms[0]])

### Function 3: `delay_embedding_persistent_homology`

A time-delay embedding of a time series (in this case, time series is the subwindow), converts the time series into a dataset in the $n-$dimensional euclidean space (via the exactly same procedure as sliding windows). Then we can simply compute the persistent homology of the corresponding Rips filtration. Different delay embedding dimensions can capture different topological features.

In [5]:
def delay_embedding_persistent_homology(subwindow, delay_embedding_dimension, delay_embedding_shift=1, maxdim=1):
    data = np.array(sliding_windows(subwindow, delay_embedding_dimension, delay_embedding_shift))
    return ripser(data, maxdim=maxdim)['dgms']

### Function 4: `get_diagram_lives`

In [6]:
def get_diagram_lives(diagram):
    return diagram[:,1]-diagram[:,0] if len(diagram)>0 else np.array([0])

### Function 5: `get_diagram_entropy`

In [7]:
def get_diagram_entropy(lives):
    if max(abs(lives)) == 0:
        return 0.
    else:
        normalizedLives = lives/sum(lives)
        return sum(-normalizedLives*np.log(normalizedLives))

### Function 6: `get_area_under_betti_curve` (for L1 and L2 norms)

In [8]:
def get_area_under_betti_curve(diagram):
    result_l1, result_l2 = 0, 0
    if len(diagram) == 0:
        return [result_l1, result_l2]
    else:
        allPts = np.concatenate([diagram[:,0],diagram[:,1]])
        birthOrDeath = np.concatenate([np.zeros(len(diagram)),np.ones(len(diagram))])
        ptsSortIndex = np.argsort(allPts)
        ptsSorted, birthOrDeathSorted = allPts[ptsSortIndex], birthOrDeath[ptsSortIndex]
        
        bettiCurveHeight = 0
        for i in range(len(ptsSorted)-1):
            if birthOrDeathSorted[i] == 0:
                bettiCurveHeight += 1
            elif birthOrDeathSorted[i] == 1:
                bettiCurveHeight += -1
                
            result_l1 += bettiCurveHeight*(ptsSorted[i+1]-ptsSorted[i])
            result_l2 += (bettiCurveHeight**2)*(ptsSorted[i+1]-ptsSorted[i])
            
        return [result_l1, np.sqrt(result_l2)]

### Function(s) 7: `get_area_under_landscape` (for L1 and L2 norms)

In [9]:
def getLandscapePoints(diagram):
    #returns the diagram points which plays a role in first layer landscape
    #gets rid of most noisy diagram points, hopefully landscape computation will last shorter
    diagram = np.unique(diagram, axis = 0) #delete duplicate rows
    diagram = diagram[np.lexsort((diagram[:,1],diagram[:,0]))] #sort by col1 then col0
    #running max of col1 equals to col1 is the good rows
    return diagram[np.maximum.accumulate(diagram[:,1]) == diagram[:,1]]

def getLandscapeReturnPoints(diagram):
    diagram = getLandscapePoints(diagram)
    diagram = diagram[diagram[:,0].argsort()]
    returnPoints = [[diagram[0,0],0]] #birth of first point
    for i in range(len(diagram)):
        #append peak which appears at (x,y) = ((d+b)/2, (d-b)/2)
        returnPoints.append([(diagram[i,1]+diagram[i,0])/2, (diagram[i,1]-diagram[i,0])/2])
        if i == len(diagram)-1:
            returnPoints.append([diagram[i,1],0])
        else:
            if diagram[i+1,0] > diagram[i,1]:
                returnPoints.append([diagram[i,1],0])
                returnPoints.append([diagram[i+1,0],0])
            else:
                returnPoints.append([(diagram[i,1]+diagram[i+1,0])/2, (diagram[i,1]-diagram[i+1,0])/2])
    return np.unique(returnPoints, axis = 0)

def l2areaUnderLine(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    result_l1 = -((x1 - x2)*(y1 + y2))/2
    result_l2 = -((x1 - x2)*(y1**2 + y1*y2 + y2**2))/3
    return [result_l1, result_l2]

def get_area_under_landscape(diagram):
    if len(diagram) == 0:
        return [0,0]
    else:
        returnPoints = getLandscapeReturnPoints(diagram)
        result_l1, result_l2 = 0, 0
        for i in range(len(returnPoints)-1):
            area_l1, area_l2 = l2areaUnderLine(returnPoints[i],returnPoints[i+1])
            result_l1 += area_l1
            result_l2 += area_l2
        return [result_l1, np.sqrt(result_l2)]

### Function 8: Putting it altogether `get_diagram_features`

In [10]:
def get_diagram_features(diagram):
    diagram = diagram[~np.any(np.isinf(diagram),axis=1)] #remove rows containing infinity (for zero-th homology class)
    lives = get_diagram_lives(diagram)
    sumLives = sum(lives)/np.sqrt(2) #feature 0
    maxLife = max(lives)/np.sqrt(2) #feature 1
    diagramEntropy = get_diagram_entropy(lives) #feature 2
    areaBettiL1, areaBettiL2 = get_area_under_betti_curve(diagram) #features 3 and 4
    areaLandscapeL1, areaLandscapeL2 = get_area_under_landscape(diagram) #features 5 and 6
    result = [sumLives, maxLife, diagramEntropy, areaBettiL1, areaBettiL2, areaLandscapeL1, areaLandscapeL2] 
    return result

### Other functions

In [11]:
# convert pd.Series([[1,2,3],[4,5,6]]) to pd.DataFrame([[1,2,3],[4,5,6]])

def pd_series_to_dataframe(series):
    # https://stackoverflow.com/a/45901074/8773741
    return pd.DataFrame.from_dict(dict(zip(series.index, series.values))).T

## Section 4: An example

Given a long time series, the below codes compute topological features.

In [12]:
time_series = np.random.randint(20, size=3000)

In [13]:
subwindow_size = 50
subwindow_shift = 10

subwindows = sliding_windows(time_series, subwindow_size, subwindow_shift)
subwindows = pd.Series(subwindows)

In [14]:
lower_level_diagrams = subwindows.map(lambda x: level_set_persistent_homology(x, upper = False))
upper_level_diagrams = subwindows.map(lambda x: level_set_persistent_homology(x, upper = True))

In [15]:
# let's try two different delay embedding dimensions
dim1, dim2 = 3, 5

rips_diagrams_dim1 = subwindows.map(lambda x: delay_embedding_persistent_homology(x, dim1))
rips_diagrams_dim2 = subwindows.map(lambda x: delay_embedding_persistent_homology(x, dim2))

In [16]:
# the rips diagrams contain two diagrams from homology classes 0 and 1. Let's split them

rips_diagrams_dim1_H0 = rips_diagrams_dim1.map(lambda x: x[0])
rips_diagrams_dim1_H1 = rips_diagrams_dim1.map(lambda x: x[1])

rips_diagrams_dim2_H0 = rips_diagrams_dim2.map(lambda x: x[0])
rips_diagrams_dim2_H1 = rips_diagrams_dim2.map(lambda x: x[1])

In [17]:
features_lower_level = lower_level_diagrams.map(get_diagram_features)
features_upper_level = upper_level_diagrams.map(get_diagram_features)

features_rips_dim1_H0 = rips_diagrams_dim1_H0.map(get_diagram_features)
features_rips_dim1_H1 = rips_diagrams_dim1_H1.map(get_diagram_features)

features_rips_dim2_H0 = rips_diagrams_dim2_H0.map(get_diagram_features)
features_rips_dim2_H1 = rips_diagrams_dim2_H1.map(get_diagram_features)

In [18]:
features_lower_level = pd_series_to_dataframe(features_lower_level)
features_upper_level = pd_series_to_dataframe(features_upper_level)

features_rips_dim1_H0 = pd_series_to_dataframe(features_rips_dim1_H0)
features_rips_dim1_H1 = pd_series_to_dataframe(features_rips_dim1_H1)

features_rips_dim2_H0 = pd_series_to_dataframe(features_rips_dim2_H0)
features_rips_dim2_H1 = pd_series_to_dataframe(features_rips_dim2_H1)

In [19]:
# now merge all features (this creates duplicate column names, we'll take care of it later)
all_subwindow_features = pd.concat([features_lower_level, features_upper_level, 
                                    features_rips_dim1_H0, features_rips_dim1_H1, 
                                    features_rips_dim2_H0, features_rips_dim2_H1], axis=1)

In [20]:
# now if the subwindow_shift is the same as window_shift
# then finding window features is quite easy actually

window_size = 140 # then how many subwindows are there in a window? Answer: 10

window_features_mean = all_subwindow_features.rolling(10).mean()
window_features_std = all_subwindow_features.rolling(10).std()

window_features = pd.concat([window_features_mean, window_features_std], axis=1).dropna()

window_features.columns = np.arange(window_features.shape[1]) #reset column names

In [21]:
window_features

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
9,99.914188,13.293607,2.471067,141.3,35.852419,89.225,23.684685,-97.934289,-0.707107,2.440248,...,4.626928,9.414970,2.304051,3.334418,0.345788,0.189828,4.715579,2.726683,1.021055,0.371421
10,101.469823,13.293607,2.490866,143.5,36.430342,88.400,23.535329,-100.338452,-0.707107,2.458448,...,4.074804,7.612124,1.832441,3.235006,0.377862,0.182781,4.574990,2.632738,1.061476,0.395625
11,102.389062,13.293607,2.495983,144.8,36.719889,88.400,23.535329,-102.176930,-0.707107,2.474227,...,3.944045,6.980378,1.667795,3.204138,0.388334,0.167502,4.531336,2.594778,1.074014,0.405304
12,105.429621,13.293607,2.508778,149.1,37.579352,88.400,23.535329,-105.783174,-0.848528,2.489464,...,5.240364,7.548730,1.806050,3.416763,0.338812,0.169997,4.832033,3.088973,0.887084,0.337072
13,108.611602,13.222897,2.524017,153.6,38.664100,87.475,23.349089,-107.692363,-0.919239,2.498299,...,5.689016,7.583974,1.811648,4.097701,0.290878,0.157632,5.795024,3.273524,0.719581,0.325961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,89.943983,12.869343,2.472732,127.2,32.426040,87.450,23.207065,-90.792511,-0.919239,2.473522,...,4.504374,4.333652,1.134470,2.002955,0.564904,0.137727,2.832606,1.753539,1.111683,0.487081
292,90.297536,13.010765,2.475140,127.7,32.886785,87.175,23.240009,-90.863221,-0.919239,2.478452,...,3.634545,4.529608,1.189249,2.072328,0.548944,0.144147,2.930714,1.755351,1.055936,0.468956
293,91.853171,13.010765,2.487391,129.9,33.620976,87.175,23.240009,-92.772410,-0.919239,2.481350,...,3.645570,4.203255,1.103955,2.276539,0.547755,0.170094,3.219512,1.754783,1.141466,0.469892
294,91.641039,12.869343,2.487200,129.6,33.751411,85.375,22.872632,-93.903781,-0.919239,2.484185,...,3.566638,4.354435,1.144669,2.001418,0.542879,0.127310,2.830432,1.658470,1.190582,0.475835


So, we ended up with 287 training examples with 84 features. 

Many columns can be correlated though.

Lastly, do not forget to scale your features.